# Week 7: Color Quantization

<font size="6"> Laboratory 4 </font> <br>
<font size="3"> Last updated August 17, 2022 </font>

## <span style="color:orange;"> 00. Content </span>

<font size="5"> Mathematics </font>
- 3 dimensional vectors
- Norm of vectors
- 1D quantization
- Euclidean distance
- 3D transforms
- Differential equations

<font size="5"> Programming Skills </font>
- Data visualization
- Functions

<font size="5"> Embedded Systems </font>
- Thonny and MicroPython

## <span style="color:orange;"> 0. Required Hardware </span>
- Microcontroller: Raspberry Pi Pico
- Breadboard
- USB connector
- NeoPixels (WS2812)
- Level shifter

<h3 style="background-color:lightblue"> Write your name and email below: </h3>

**Name:** me 

**Email:** me @purdue.edu

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
%matplotlib inline

## <span style="color:orange;"> 1. Grayscale/Desaturation </span>

"Grayscaling" an image, also called desaturation, is when all the color in the image is transformed to appear as a shade of gray without apparent color. Specifically, these shades represent the intensity information of the light. This can be done for a variety of reasons, from taking up less storage space to aesthetics.

When coding this transformation, there are a multitude of different methods that can be used, many of which involve calculus principles. However, some of the mathematics that may be assumed to work don't produce accurate results. Even if the resulting image looks like its been grayscaled, some of the darks or lights might be too pronounced, or the shadows might be outlined, causing the blending to look awkward.

Let's try some different methods to produce new images.

In [ ]:
# function to show original and grayscaled images
def showimage(img, gray_img = [[0]]):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
    ax[0].imshow(np.array(img,dtype=np.uint8))
    ax[0].axis('off')   
    ax[1].imshow(np.array(gray_img), cmap='gray', vmin=0, vmax=255)
    ax[1].axis('off')            
    plt.show()

## Magnitude

One of the methods that may be assumed to work is calculating the length, or magnitude of the image array. The magnitude of a vector can be used to find the unit vector, which has a length of 1. Similarly, grayscaling works by transforming the pixels so they only represent the intensity/brightness information of the light. Intensity is shown in an image through blacks, whites, and grays, which results in the image looking to be shades of these three "colors." By taking the magnitude and applying it to the original image array values, it can be reasoned that the resultant values would only represent a portion of the image's characteristics.

### <span style="color:red"> Exercise 1 </span>

What happens if we replace the RGB vector for a pixel with the scaled norm of that vector? That means we need to calculate the magnitude of the vector $(R,G,B)$ and then rescale so that the grayscale values are between 0-255. Fill in the function that finds this new kind of grayscale image. Display the result.

Download the picture we'll be using for this lab: [mnms.jpg](https://github.com/TheDataScienceLabs/DataLab_Multivariate_Calculus/blob/main/book/labs/1_Color_Labs/3_Color_Quantization/mnms.jpg).

<h3 style="background-color:lightblue"> Write Answers for Exercise 1 Below </h3>

In [ ]:
img = Image.open('mnms.jpg')

def magnitude_method(img):
    img = np.array(img, dtype='float64') # we have to change the datatype so the calculations are with floating point numbers
    pass

showimage(img,magnitude_method(img))

At first glance, this might seem like a perfectly reasonable solution, since everything is grayed over and the highlights and shadows match the original picture. However, some of colors are actually completely off, such as the red and green colors being almost indistinguishable and blue colors being too light. These issues might seem minimal, with some of them not even being noticeable until compared with a more proper solution (which will be done later), but they are nonetheless issues, meaning this is not the best solution for grayscaling an image.

### RGB Versus HSV

These issues may be more apparent when using the HSV (hue, saturation, value) values for the image. You may have seen HSV value options when using a printer or image editing software. While RGB defines color in terms of the combination of the primary, colors red, green, and blue, HSV deals with lighting and tints. HSV is more sensitive to changes in lighting and is more helpful when making detailed and realistic modifications to an image. In this course, we'll be primarily using RGB since it is more common for smaller changes like we're doing and is also more well-known as a concept.

However, HSV is actually the closest scheme to how humans perceive color, in comparison to RGB and CMYK (cyan, magenta, yellow, key; another color scheme). Hue is the portion of the model and is expressed as a number from 0 to 360 degrees, with primary colors falling in certain degree ranges. Saturation describes the amount of gray in a color and ranges from 0 to 100%, where the closer it gets to 0%, the more gray a color is. Value, also called brightness in some softwares (HSB), describes the intensity of a color and is also defined from 0 to 100%, with 0% being black/the darkest and 100% being white/the brightest, meaning it reveals the most color. The Python Image Library (PIL) uses 8-bit integers to represent hue, saturation, and value, so each channel ranges between 0-255.

In [ ]:
img_rgb = Image.open('mnms.jpg')
showimage(img_rgb,magnitude_method(img_rgb)) # original image vs RGB magnitude

img_hsv = img.convert('HSV')
showimage(img,magnitude_method(img_hsv)) # original image vs HSV magnitude

## Averaging

Other than magnitude, you may also think that taking the average of the RGB values and applying it to the image would produce a grayscaled image. The goal with grayscaling is to diminish the color values to just their intensities, so, similar to magnitude, it may make sense to try and simply find what the "middle" of the colors is. It can also be considered a form of mixing, which is a way that gray-based colors can be found if you think about it in terms of mixing paint.

### <span style="color:red"> Exercise 2 </span>

What happens if we replace the RGB vector for a pixel with the average of the 3 vector values? Use the same M\&M's picture from the previous exercise. Compare the results with the magnitude method.

<h3 style="background-color:lightblue"> Write Answers for Exercise 2 Below </h3>

In this case, we will not be taking the average of the HSV values since these values represent different characteristics, unlike how RGB is representative of the combination of the values. In other words, averaging HSV values does not have a physical interpretation. 

## Using a Specific Vector

As mentioned before, taking the magnitude or average of the colors doesn't produce a well-grayscaled image for multiple reasons. So what do people normally do when trying to grayscale an image? Again, there are many methods that can be used, but one of the most common, and simplest, ways is to take the dot product of the RGB vector at each pixel and a specific vector that has the express purpose of manipulating the red, green, and blue values in an image to be gray. The specific vector you see most often is `[0.2989, 0.5870, 0.1140]`; however there are a few other vectors that produce properly grayscaled images, such as `[0.2126, 0.7512, 0.0722]`. Rounding these vectors to the tenth place is also used relatively often in coding, producing `[0.3, 0.6, 0.1]`.

### <span style="color:red"> Exercise 3 </span>

__Part 1:__ The averaging method in Exercise 2 is equivalent to grayscaling with what specific vector?

__Part 2:__ Use the vector `[0.2989, 0.5870, 0.1140]` to grayscale the image of M\&M's.

<h3 style="background-color:lightblue"> Write Answers for Exercise 3 Below </h3>

If you compare the resulting grayscaled image to the other methods used, you can better see that the other methods aren't producing the correct images. The correctly grayscaled image has more subdued tones and the colors blend together better. Besides this, the image also has more definition. The other methods produced images where the objects looked nearly flat or 2-dimensional even though they're 3-dimensional objects, which most likely occurs due to a lack of proper blending.


As can be seen, the colors are a little bit different, unlike what happened with the magnitude method. However, in the HSV grayscaled image, the different colors are actually a bit easier to pick out, especially in the back of the picture where the lighting caused by the flash is less prevalent. As mentioned before, HSV is more sensitive to lighting differences, which is most likely why the colors are easier to pick apart in the darker part of the picture as compared to the lighter part of the picture.

<h3 style="color:green;"><left> Sandbox </left></h3>
<span style="color:green;"><left> </left></span>
Try playing around with the vector values to see what happens to the resulting image. For example, try making one value much larger or smaller than the other two, making all the values extremely large or small, etc. You can get some super weird looking images from doing this! Or look at what the grayscaled image of the HSV looks like in comparison to see what color intensities convert better than others.

## <span style="color:orange;"> 2. Quantization </span>

We mentioned before that we have $256 \times 256 \times 256 = 16,777,216$ colors to work with since there are 256 integers to choose from for a red, green, and blue channel value. This is one form of color quantization since there are infinitely many colors and we've reduced that down to about 16.7 million. In many applications, further quantization is necessary. Quantization is used for image compression by reducing the information of an image so that it requires less storage, and some devices like phones or printers only support a certain palette of colors. 

Grayscaling, like the transformations you were performing in the first section of this lab, can be considered a form of quantization as you were taking a large collection of different values and converting them to be a smaller set of values.

Now, suppose we are only able to use 10 gray values instead of the usual 256. We can do this with a quantization algorithm.
Uniform quantization is a simple type of quantization where the quantization levels are equally spaced. Here's how the uniform quantization function looks.


In [ ]:
num_levels=10
x = np.linspace(0,255,num_levels+1)
y = np.linspace(0,255,num_levels)
  
plt.step(x, np.append(y,y[-1]), where='post')
plt.xlabel('Grayscale Value')
plt.ylabel('Quantized Value')
plt.title('Uniform Quantization in 1D')
plt.show()

### <span style="color:red"> Exercise 4 </span>

In your own words, explain how to read the graph above as if you were talking to a friend who has never learned about color science. 

<h3 style="background-color:lightblue"> Write Answers for Exercise 4 Below </h3>

### <span style="color:red"> Exercise 5</span>

Write a function that converts an image to grayscale and then uniformly quantizes the gray image with $k$ levels. Show your results when $k=2,10,24,$ and $100$.

<h3 style="background-color:lightblue"> Write Answers for Exercise 5 Below </h3>

We can also quantize color images. Let's define our own palette of colors and map each to pixel to 1 of 4 options based on Euclidean distance.
The distance between the two colors $(R_1,G_1,B_1)$ and $(R_2,G_2,B_2)$ is $
    d = \sqrt{ (R_1-R_2)^2 + (G_1-G_2)^2 + (B_1-B_2)^2 }.$ 
    
### <span style="color:red"> Exercise 6 </span>

**Part 1** 

Choose your own palette of at least 4 colors and write a function that quantizes a color image (of your chosing) to your palette based on the shortest Euclidean distance among the chosen colors. For example, if the palette of four colors is (0,0,10), (41,200,0), (36,0,85), and (40,40,40), then to quantize a pixel in the original image with RGB values (1,1,9) compute the 4 distances between (1,1,9) and (0,0,10); (1,1,9) and (41,200,0); etc. Replace the original value (1,1,9) with the color (0,0,10) from the palette since that color is the shortest distance from the original color.

<h3 style="background-color:lightblue"> Write Answers for Exercise 6 Part 1 Below </h3>

**Part 2** 

We've introduced quite a few color spaces like RGB, HSV, and CMYK. Similarly, another color space named the CIE L\*a\*b\* space is an important color space in color science and is widely accepted to model human perception of color as accurately as possible.

Quantize your image from Part 1 again but this time based on the shortest Euclidean distance in the CIE L\*a\*b\* space. That is, transform the RGB vectors to L\*a\*b\* vectors and then compute the distance between the two L\*a\*b\* vectors. See the example below on how to convert from RGB to CIE L\*a\*b\*.

<h3 style="background-color:lightblue"> Write Answers for Exercise 6 Part 2 Below </h3>

In [ ]:
rgb = np.array([0,0,255])
lab = cv2.cvtColor( np.uint8([[rgb]] ), cv2.COLOR_RGB2LAB)[0][0].astype(float)
lab[0] *= 100/255
lab[1] -= 128
lab[2] -= 128
print('RGB {} ---> L*a*b* {}'.format(rgb,lab))

**Part 3** 

Do you notice any differences in the two quantization methods? What if you add more or less colors to your palette?

<h3 style="background-color:lightblue"> Write Answers for Exercise 6 Part 3 Below </h3>

There are methods for adaptive color quantization, which means that the palette of colors changes based on the image. We won't explore them for now, but here's an example where the 'best' 15 colors are chosen.

In [ ]:
img = Image.open('mnms.jpg')
quant_img = img.quantize(colors=15)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ax[0].imshow(img)
ax[0].axis('off')            
ax[1].imshow(quant_img)
ax[1].axis('off')
plt.show()

## <span style="color:orange;"> 3. Gamma Correction </span>

Consider the game of "guess how many jellybeans (or other small objects) are in the jar". This game would be very uninteresting if there were only 4 jellybeans in the jar. We would have a much harder time visually distinguishing between 140 and 141 jellybeans, but there is some threshold where, say, 180 jellybeans looks noticeably different from 140 jellybeans. If you're a fan of psychology, you may have heard of [Weber's Law](https://en.wikipedia.org/wiki/Weber%E2%80%93Fechner_law), which quantifies human perception in response to a stimulus like an increase in jellybeans. Weber's Law suggests that this threshold is proportional to the relative stimulus increment. In other words, if $dV$ is a just perceivable difference in the amount of jellybeans $J$, then for a constant $\alpha$, $
    dV = \alpha \frac{dJ}{J}.$
Integrating both sides, we get $
    V = \alpha \ln(J).$
Weber's Law also applies to how we perceive light and color. The relationship we just derived suggests that we should quantize color levels logarithmically instead of uniformly (like in Exercise 5) so the levels are farther apart as the value increases. 

### <span style="color:red"> Exercise 7</span>

Follow the wiring directions from Lab 2 to connect at least one NeoPixel to your Pico.
Write a script that displays Purdue Old Gold (RGB = (206, 184, 136)) on the NeoPixel(s).

It doesn't look quite right. That's because according to Weber's Law we need to adjust the brightness levels for the NeoPixel to be logarithmic. This process is called *gamma correction*. The new, gamma-corrected RGB values are $
    R_{new} = 255(R/255)^\gamma, \quad G_{new} = 255(G/255)^\gamma, \quad B_{new} = 255(B/255)^\gamma.$
Typically, $\gamma=2.2$ for cameras and scanners, but the best gamma value depends on the hardware.

Gamma correct the Purdue Old Gold RGB values for different values $1 \leq \gamma \leq 10$ and show these colors on the NeoPixels. What does Purdue gold look like now? Report your findings. 

<h3 style="background-color:lightblue"> Write Answers for Exercise 7 Below </h3>

## <span style="color:green"> Reflection </span>

__1. What parts of the lab, if any, do you feel you did well? <br>
2. What are some things you learned today? <br>
3. Are there any topics that could use more clarification? <br>
4. Do you have any suggestions on parts of the lab to improve?__

<h3 style="background-color:lightblue"> Write Answers for the Reflection Below </h3>